In [1]:
import numpy as np
import multiprocessing as mp
import pandas as pd
import sys
sys.path.append('utils/')
import warnings

from utils import s3_tools
import os
os.environ['PATH'] += ':/home/csci5980/piehl008/software/ffmpeg/bin/'

%cd ~/csci5980

/panfs/roc/groups/5/csci5980/piehl008/csci5980


In [2]:
# GLOBAL VARIABLES
SR = 44100

# number of audio files per tfrecord file
TF_GROUP_SIZE = 150

# length threshold, clip must have at least 75% non-zeros to append zeros
LENGTH_THRESHOLD = 0.75

In [3]:
# set track ids

from pathlib import Path

def get_track_id_from_file(filename):
    return int(Path(filename).stem)

data_subset = pd.read_csv('genre_subsets.txt', names=['raw_audio_file'])

data_subset['track_id'] = data_subset['raw_audio_file'].map(get_track_id_from_file)
data_subset = data_subset.set_index('track_id')
data_subset

,raw_audio_file
track_id,
20,fma_large/000/000020.mp3
26,fma_large/000/000026.mp3
30,fma_large/000/000030.mp3
46,fma_large/000/000046.mp3
48,fma_large/000/000048.mp3
...,...
155302,fma_large/155/155302.mp3
155303,fma_large/155/155303.mp3
155304,fma_large/155/155304.mp3


In [4]:
# get track and genre data
client = s3_tools.get_s3_client()
genres = s3_tools.load_csv_from_s3(client, 'fma-dataset', 'fma-metadata/genres.csv',
                                   index_col=0)

genres_subset = genres[genres['title'].isin(['Ambient Electronic',
                                             'Singer-Songwriter',
                                             'Garage'])]
print(genres_subset)
genre_codes_subset = genres_subset.index
genre_codes_subset

          #tracks  parent               title  top_level
genre_id                                                
42           5723      15  Ambient Electronic         15
85           3548      12              Garage         12
103          4162      17   Singer-Songwriter         17


Int64Index([42, 85, 103], dtype='int64', name='genre_id')

In [5]:
def get_s3_path(filename):
    audio_dirs = Path(filename).parts[-2:]
    return str(Path('raw-audio') / Path(*audio_dirs))

data_subset.loc[:, 'raw_audio_file'] = data_subset['raw_audio_file'].apply(get_s3_path)
data_subset

,raw_audio_file
track_id,
20,raw-audio/000/000020.mp3
26,raw-audio/000/000026.mp3
30,raw-audio/000/000030.mp3
46,raw-audio/000/000046.mp3
48,raw-audio/000/000048.mp3
...,...
155302,raw-audio/155/155302.mp3
155303,raw-audio/155/155303.mp3
155304,raw-audio/155/155304.mp3


In [6]:
def parse_string_to_int_list(string_list):
    try:
        return [int(x) for x in string_list.rstrip(']').lstrip('[').split(', ')]
    except:
        return [np.nan]

def filter_to_genre_subset(genre_list):
    if any([x in genre_codes_subset for x in genre_list]):
        return True
    else:
        return False

def filter_out_wrong_genres(genre_list):
    return [x for x in genre_list if x in genre_codes_subset]

tracks = s3_tools.load_csv_from_s3(client, 'fma-dataset', 'fma-metadata/tracks.csv',
                                   index_col=0, header=[0, 1])
tracks.loc[:, ('track', 'genres_all')] = tracks.loc[:, ('track', 'genres_all')].apply(parse_string_to_int_list)
tracks = tracks[tracks.loc[:, ('track', 'genres_all')].apply(filter_to_genre_subset)]
tracks.loc[:, ('track', 'genres_all')] = tracks.loc[:, ('track', 'genres_all')].apply(filter_out_wrong_genres)
tracks = tracks[tracks.loc[:, ('track', 'genres_all')].apply(len) == 1]
tracks.loc[:, ('track', 'genres_all')] = tracks.loc[:, ('track', 'genres_all')].apply(lambda x: x[0])
track_genre_codes = pd.DataFrame({'genre_code': tracks.loc[:, ('track', 'genres_all')]})
track_genre_codes = track_genre_codes.join(data_subset)

In [7]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
track_genre_codes['genre_code'] = label_encoder.fit_transform(track_genre_codes['genre_code'])

encoder_df = pd.DataFrame.from_dict({'code': label_encoder.transform(label_encoder.classes_),
                                     'label': label_encoder.classes_})
encoder_df.to_csv('~/csci5980/genre_encodings.csv')

In [8]:
from data.load_data import augment_waveform, mu_transform
import numpy as np
client = s3_tools.get_s3_client()

def get_waveforms(raw_file, augment=True):
    """Gets waveforms split into 1 second clips for file
    
    Returns (list of waveforms, list of augmented waveforms)
    """
    try:
        warnings.filterwarnings("ignore")
        waveform, true_sr = s3_tools.load_raw_audio_from_s3(client, 'fma-dataset',
                                                        raw_file, sr=SR)
        assert true_sr == SR, 'Sampling rate not met'
        warnings.filterwarnings('default')
    except:
        warnings.warn('Unable to create waveform for {}'.format(raw_file))
        return []

    waveform = np.trim_zeros(waveform)
    split_waveform = np.split(waveform, np.arange(SR, len(waveform), SR))
    if len(split_waveform[-1]) / float(SR) < LENGTH_THRESHOLD:
        split_waveform = split_waveform[:-1]
    else:
        split_waveform[-1] = np.pad(split_waveform[-1], (0, SR - len(split_waveform[-1])),
                                    'constant', constant_values=(0, 0))
    if augment:
        split_augmented = [augment_waveform(s, sr=SR) for s in split_waveform]
        return mu_transform(split_waveform), mu_transform(split_augmented)

    return mu_transform(split_waveform), mu_transform(split_waveform)
wf, wf_aug = get_waveforms(track_genre_codes['raw_audio_file'].iloc[0], augment=True)

In [9]:
# shuffle data, split into groups
track_genre_codes_shuffled = track_genre_codes.sample(frac=1, replace=False, random_state=5980)
record_groups = np.array_split(track_genre_codes_shuffled.index,
                               len(track_genre_codes_shuffled) // TF_GROUP_SIZE)
record_groups

/home/csci5980/piehl008/.conda/envs/wavenet/lib/python3.7/site-packages/jupyter_client/jsonutil.py:66: DeprecationWarning: Interpreting naive datetime as local 2020-04-29 22:12:49.047494. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)


[Int64Index([114508, 143133, 128896, 111317,  77225,  85211, 145005,  59223,
               3541,  47728,
             ...
              12440,  87385, 121797,  67428,  66465, 111039,  40647,  52366,
              79371,  93363],
            dtype='int64', name='track_id', length=151),
 Int64Index([ 70555,  43894, 115561,  91368, 148216,  76712,  93361, 101943,
             144956, 144953,
             ...
             137206,  44120,  54199, 138900,  99645, 142637, 131096, 103764,
             116142,  68683],
            dtype='int64', name='track_id', length=151),
 Int64Index([ 17420, 110924, 111184,  63683,  44455,  99656,  55821, 140510,
             138199,  39503,
             ...
              61726,   9099, 105488,  14272,  40409, 136069, 119931, 138247,
              57658, 147088],
            dtype='int64', name='track_id', length=151),
 Int64Index([ 39336,  90680,  14188,  66179,  39341,  90374,  87069,  93389,
             113162,  13103,
             ...
             108

In [10]:
record_group = track_genre_codes.loc[record_groups[0], :]

def get_training_waveforms(raw_audio_file):
    print(raw_audio_file)
    return get_waveforms(raw_audio_file, augment=True)

len(record_groups)

/home/csci5980/piehl008/.conda/envs/wavenet/lib/python3.7/site-packages/jupyter_client/jsonutil.py:66: DeprecationWarning: Interpreting naive datetime as local 2020-04-29 22:12:49.574531. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)


89

In [11]:
"""
The following functions are modeled exactly on the guide found at 
https://www.tensorflow.org/tutorials/load_data/tfrecord
"""
from tqdm import tqdm
import pickle

def _int64_feature(value):
    """Returns an int64_list from a bool / enum / int / uint."""
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

def _int64_list_feature(value):
    """Returns an int64_list from a bool / enum / int / uint."""
    return tf.train.Feature(int64_list=tf.train.Int64List(value=value))


def serialize(genre_code, waveform, augmented_waveform):
    """
    Creates a tf.Example message ready to be written to a file.
    """
    feature = {
        'genre_code': tf.io.serialize_tensor(genre_code).numpy(),
        'waveform': tf.io.serialize_tensor(waveform).numpy(),
        'augmented_waveform': tf.io.serialize_tensor(augmented_waveform).numpy()
    }

    serialized = pickle.dumps(feature)
    return serialized


def tf_serialize(f0, f1, f2):
    """Serializes elements f0, f1, f2 for writing to tfrecord file

    """
    tf_string = tf.py_function(
        serialize,
        (f0, f1, f2),
        tf.string)
    return tf.reshape(tf_string, ())


def write_record(genre_codes, waveforms, augmented_waveforms, filename):
    """Writes out record file with top genres and spectrograms

    Args:
      genre_codes (list of int): Top genre codes
      waveforms (list of np.ndarray): waveform array
      augmented_waveforms (list of np.ndarray): augmented waveform array
      filename (str): Filename to write tfrecord to
    """
    features_dataset = tf.data.Dataset.from_tensor_slices((genre_codes,
                                                           waveforms,
                                                           augmented_waveforms))
    serialized_features_dataset = features_dataset.map(tf_serialize,
                                                       num_parallel_calls=tf.data.experimental.AUTOTUNE)
    record_sizes = []
    with open(filename, 'wb') as writer:
        for element in serialized_features_dataset:
            byte_string = element.numpy()
            record_sizes.append(len(byte_string))
            writer.write(byte_string)
    return record_sizes

/home/csci5980/piehl008/.conda/envs/wavenet/lib/python3.7/site-packages/jupyter_client/jsonutil.py:66: DeprecationWarning: Interpreting naive datetime as local 2020-04-29 22:12:50.240444. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)


In [12]:
import random
from itertools import chain
import tensorflow as tf

def make_records(base_dir, group_dataset, idx):
    base_dir = Path(base_dir)
    assert base_dir.exists()

    record_base_name = base_dir / ('record_' + str(idx))
    record_name = record_base_name.with_suffix('.pickle')
    record_metadata_name = record_base_name.with_suffix('.csv')

    if record_name.exists() and record_metadata_name.exists():
        return
    
    record_name = str(record_name)
    record_metadata_name = str(record_metadata_name)
    record_group = track_genre_codes.loc[group_dataset, :]

    out_waves = []
    out_waves = map(get_training_waveforms, list(record_group['raw_audio_file']))

    out_waves = [wave_list for wave_list in out_waves if len(wave_list) == 2]
    waveforms = [[wf for wf in wave_list[0]] for wave_list in out_waves]
    track_ids = [[track_id for wave in wave_list]
                  for track_id, wave_list in zip(record_group.index, waveforms)]
    augmented_waveforms = [[wf_aug for wf_aug in wave_list[1]] for wave_list in out_waves]
    waveform_lengths = [[len(wf) for wf in wave_list] for wave_list in waveforms]
    genre_codes = [[code for wave in wave_list]
                   for code, wave_list in zip(record_group['genre_code'], waveforms)]
    records = [[str(Path(record_name).stem) for wave in wave_list] for wave_list in waveforms]

    track_ids = list(chain(*track_ids))
    waveforms = list(chain(*waveforms))
    augmented_waveforms = list(chain(*augmented_waveforms))
    genre_codes = list(chain(*genre_codes))
    tfrecords = list(chain(*records))

    # shuffle dataset
    zipped_list = list(zip(track_ids, waveforms, augmented_waveforms,
                           genre_codes, tfrecords))
    random.shuffle(zipped_list)
    shuffled_track_ids, shuffled_waveforms, shuffled_augmented_waveforms, \
        shuffled_genre_codes, shuffled_tfrecords = zip(*zipped_list)

    record_metadata = pd.DataFrame.from_dict({
          'track_id': shuffled_track_ids,
          'genre_code': shuffled_genre_codes,
          'tfrecord': shuffled_tfrecords
    })
    
    record_sizes = write_record(list(shuffled_genre_codes), list(shuffled_waveforms),
                                list(shuffled_augmented_waveforms), record_name)
    record_metadata['record_size'] = record_sizes
    record_metadata.to_csv(record_metadata_name, index=False)

/home/csci5980/piehl008/.conda/envs/wavenet/lib/python3.7/site-packages/jupyter_client/jsonutil.py:66: DeprecationWarning: Interpreting naive datetime as local 2020-04-29 22:12:51.108930. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)


In [13]:
%cd '/scratch.global/piehl008/'

/scratch.global/piehl008


/home/csci5980/piehl008/.conda/envs/wavenet/lib/python3.7/site-packages/jupyter_client/jsonutil.py:66: DeprecationWarning: Interpreting naive datetime as local 2020-04-29 22:12:51.831701. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)


In [ ]:
from subprocess import Popen, PIPE

base_temp_dir = '/scratch.global/piehl008/pickled_data/'
uploading_processes = []

for i, group in tqdm(enumerate(record_groups),
                     desc='Writing tfrecords',
                     total=len(record_groups)):
    make_records(base_temp_dir, group, i)
    local_files = 'pickled_data/record_' + str(i) + '*'
    s3_files = 's3://fma-dataset/pickled_data/'
    uploading_processes.append(Popen('s3cmd put ' + local_files + ' ' + s3_files, shell=True))

[proc.wait() for proc in uploading_processes]

/home/csci5980/piehl008/.conda/envs/wavenet/lib/python3.7/site-packages/jupyter_client/jsonutil.py:66: DeprecationWarning: Interpreting naive datetime as local 2020-04-29 22:12:52.775708. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)
Writing tfrecords:   0%|          | 0/89 [00:00<?, ?it/s]

raw-audio/039/039336.mp3
raw-audio/090/090680.mp3
raw-audio/014/014188.mp3
raw-audio/066/066179.mp3
raw-audio/039/039341.mp3
raw-audio/090/090374.mp3
raw-audio/087/087069.mp3
raw-audio/093/093389.mp3
raw-audio/113/113162.mp3
raw-audio/013/013103.mp3
raw-audio/108/108046.mp3
raw-audio/058/058355.mp3
raw-audio/111/111096.mp3
raw-audio/060/060244.mp3
raw-audio/039/039505.mp3
raw-audio/009/009045.mp3
raw-audio/099/099054.mp3
raw-audio/064/064453.mp3
raw-audio/115/115835.mp3
raw-audio/136/136384.mp3
raw-audio/107/107142.mp3
raw-audio/105/105916.mp3
raw-audio/039/039991.mp3
raw-audio/119/119446.mp3
raw-audio/065/065191.mp3
raw-audio/133/133017.mp3
raw-audio/054/054124.mp3
raw-audio/083/083083.mp3
raw-audio/066/066629.mp3
raw-audio/106/106545.mp3
raw-audio/112/112013.mp3
raw-audio/077/077405.mp3
raw-audio/004/004300.mp3
raw-audio/148/148188.mp3
raw-audio/107/107525.mp3
raw-audio/024/024988.mp3
raw-audio/108/108366.mp3
raw-audio/087/087355.mp3
raw-audio/103/103744.mp3
raw-audio/047/047604.mp3


Writing tfrecords:   4%|▍         | 4/89 [05:26<1:55:43, 81.68s/it]

raw-audio/097/097586.mp3
raw-audio/104/104032.mp3
raw-audio/050/050297.mp3
raw-audio/053/053805.mp3
raw-audio/086/086665.mp3
raw-audio/111/111704.mp3
raw-audio/138/138412.mp3
raw-audio/122/122120.mp3
raw-audio/051/051048.mp3
raw-audio/075/075889.mp3
raw-audio/081/081174.mp3
raw-audio/071/071915.mp3
raw-audio/030/030422.mp3
raw-audio/095/095467.mp3
raw-audio/070/070698.mp3
raw-audio/064/064777.mp3
raw-audio/093/093394.mp3
raw-audio/081/081839.mp3
raw-audio/136/136072.mp3
raw-audio/084/084339.mp3
raw-audio/036/036747.mp3
raw-audio/086/086471.mp3
raw-audio/146/146902.mp3
raw-audio/004/004279.mp3
raw-audio/046/046795.mp3
raw-audio/091/091381.mp3
raw-audio/099/099378.mp3
raw-audio/118/118524.mp3
raw-audio/048/048895.mp3
raw-audio/144/144413.mp3
raw-audio/129/129453.mp3
raw-audio/037/037340.mp3
raw-audio/071/071524.mp3
raw-audio/107/107900.mp3
raw-audio/099/099473.mp3
raw-audio/152/152532.mp3
raw-audio/134/134004.mp3
raw-audio/063/063890.mp3
raw-audio/136/136299.mp3
raw-audio/073/073119.mp3


Writing tfrecords:   6%|▌         | 5/89 [12:01<4:05:53, 175.63s/it]

...

raw-audio/012/012313.mp3
raw-audio/138/138599.mp3
raw-audio/072/072696.mp3
raw-audio/044/044754.mp3
raw-audio/039/039997.mp3
raw-audio/113/113159.mp3
raw-audio/147/147350.mp3
raw-audio/046/046013.mp3
raw-audio/104/104037.mp3
raw-audio/067/067835.mp3
raw-audio/041/041465.mp3
raw-audio/060/060900.mp3
raw-audio/138/138406.mp3
raw-audio/042/042357.mp3
raw-audio/029/029546.mp3
raw-audio/013/013453.mp3
raw-audio/101/101618.mp3
raw-audio/080/080493.mp3
raw-audio/069/069631.mp3
raw-audio/097/097097.mp3
raw-audio/099/099419.mp3
raw-audio/051/051229.mp3
raw-audio/001/001341.mp3
raw-audio/146/146966.mp3
raw-audio/091/091883.mp3
raw-audio/017/017418.mp3
raw-audio/023/023570.mp3
raw-audio/066/066501.mp3
raw-audio/091/091328.mp3
raw-audio/115/115562.mp3
raw-audio/092/092583.mp3
raw-audio/042/042735.mp3
raw-audio/003/003668.mp3
raw-audio/141/141513.mp3
raw-audio/125/125000.mp3
raw-audio/114/114436.mp3
raw-audio/044/044175.mp3
raw-audio/124/124227.mp3
raw-audio/105/105560.mp3
raw-audio/033/033199.mp3


Writing tfrecords:  81%|████████  | 72/89 [7:49:43<1:54:59, 405.83s/it]

raw-audio/066/066542.mp3
raw-audio/152/152096.mp3
raw-audio/009/009136.mp3
raw-audio/019/019336.mp3
raw-audio/073/073433.mp3
raw-audio/106/106728.mp3
raw-audio/054/054633.mp3
raw-audio/065/065084.mp3
raw-audio/119/119718.mp3
raw-audio/138/138383.mp3
raw-audio/034/034926.mp3
raw-audio/003/003953.mp3
raw-audio/107/107038.mp3
raw-audio/086/086065.mp3
raw-audio/094/094087.mp3
raw-audio/097/097680.mp3
raw-audio/012/012953.mp3
raw-audio/018/018188.mp3
raw-audio/112/112134.mp3
raw-audio/111/111643.mp3
raw-audio/080/080554.mp3
raw-audio/054/054694.mp3
raw-audio/048/048065.mp3
raw-audio/042/042861.mp3
raw-audio/052/052365.mp3
raw-audio/085/085909.mp3
raw-audio/067/067425.mp3
raw-audio/019/019280.mp3
raw-audio/145/145538.mp3
raw-audio/092/092491.mp3
raw-audio/151/151307.mp3
raw-audio/138/138072.mp3
raw-audio/038/038507.mp3
raw-audio/064/064283.mp3
raw-audio/042/042778.mp3
raw-audio/105/105888.mp3
raw-audio/104/104366.mp3
raw-audio/085/085251.mp3
raw-audio/065/065859.mp3
raw-audio/116/116765.mp3


Writing tfrecords:  82%|████████▏ | 73/89 [7:56:23<1:47:45, 404.07s/it]

raw-audio/105/105660.mp3
raw-audio/122/122181.mp3
raw-audio/104/104255.mp3
raw-audio/074/074685.mp3
raw-audio/135/135964.mp3
raw-audio/070/070561.mp3
raw-audio/091/091515.mp3
raw-audio/105/105861.mp3
raw-audio/042/042858.mp3
raw-audio/107/107041.mp3
raw-audio/104/104653.mp3
raw-audio/105/105454.mp3
raw-audio/066/066415.mp3
raw-audio/112/112596.mp3
raw-audio/086/086584.mp3
raw-audio/125/125186.mp3
raw-audio/129/129692.mp3
raw-audio/097/097124.mp3
raw-audio/105/105485.mp3
raw-audio/080/080000.mp3
raw-audio/114/114786.mp3
raw-audio/104/104285.mp3
raw-audio/065/065803.mp3
raw-audio/100/100753.mp3


In [28]:
track_genre_codes.loc[group[35], :]

genre_code                               0
raw_audio_file    raw-audio/151/151920.mp3
Name: 151920, dtype: object

In [2]:
def _parse_function(example_proto):
    """Parses the input tf.Example proto using the dictionary above.

    Args:
      example_proto (tf.Example proto)
      spectrogram_width (int): Expected width of spectrogram

    Returns:
      parsed tfrecord message
    """
    feature_description = {
        'genre_code': tf.io.FixedLenFeature([], tf.int64, default_value=0),
        'waveform': tf.io.FixedLenSequenceFeature([44100], tf.int64, allow_missing=True),
        'augmented_waveform': tf.io.FixedLenSequenceFeature([44100], tf.int64, allow_missing=True)
    }

    parsed = tf.io.parse_single_example(example_proto, feature_description)

    return (parsed['genre_code'], tf.expand_dims(parsed['waveform'], axis=-1),
            tf.expand_dims(parsed['augmented_waveform'], axis=-1))

In [32]:
!s3info

/home/csci5980/piehl008/.conda/envs/wavenet/lib/python3.7/site-packages/jupyter_client/jsonutil.py:66: DeprecationWarning: Interpreting naive datetime as local 2020-04-29 20:30:21.939726. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)


Your current data usage in Second Tier Storage is 114.09 GiB.

Access key: 38XZS5ACPIDWZXKLDS49
Secret key: a0hLS8VGRrh3yIq8nl89KFj6xA7QjJRgsNaL3MAX

Sharing address: piehl008@umn.edu
Tier 2 username: uid=75217


In [33]:
import os
os.environ['AWS_ACCESS_KEY_ID'] = '38XZS5ACPIDWZXKLDS49'
os.environ['AWS_SECRET_ACCESS_KEY'] = 'a0hLS8VGRrh3yIq8nl89KFj6xA7QjJRgsNaL3MAX'

/home/csci5980/piehl008/.conda/envs/wavenet/lib/python3.7/site-packages/jupyter_client/jsonutil.py:66: DeprecationWarning: Interpreting naive datetime as local 2020-04-29 20:30:59.155699. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)


In [37]:
import tensorflow as tf

def deserialize(raw_bytes):
    serialized = pickle.loads(raw_bytes.numpy())
    genre_code = tf.io.parse_tensor(serialized['genre_code'], tf.int32)
    waveform = tf.io.parse_tensor(serialized['waveform'], tf.int32)
    augmented_waveform = tf.io.parse_tensor(serialized['augmented_waveform'], tf.int32)
    return genre_code, waveform, augmented_waveform

def tf_deserialize(raw_bytes):
    genre_code, waveform, augmented_waveform = tf.py_function(
        deserialize,
        [raw_bytes],
        (tf.int32, tf.int32, tf.int32))
    return genre_code, waveform, augmented_waveform

rawdata = tf.data.FixedLengthRecordDataset(['s3://fma-dataset/pickled_data/record_0.pickle',
                                            's3://fma-dataset/pickled_data/record_1.pickle'], 80119)
dataset = rawdata.map(tf_deserialize)
#parsed_dataset = rawdata.map(_parse_function)
for el in dataset.batch(500):
    print(el)
    break

/home/csci5980/piehl008/.conda/envs/wavenet/lib/python3.7/site-packages/jupyter_client/jsonutil.py:66: DeprecationWarning: Interpreting naive datetime as local 2020-04-29 20:32:48.579752. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)


(<tf.Tensor: shape=(360,), dtype=int32, numpy=
array([2, 1, 1, 0, 2, 0, 2, 0, 1, 0, 0, 2, 0, 2, 0, 1, 0, 2, 0, 2, 2, 0,
       2, 2, 0, 1, 2, 2, 0, 0, 2, 0, 0, 0, 2, 2, 1, 2, 1, 2, 1, 1, 2, 0,
       2, 2, 0, 2, 2, 1, 0, 2, 0, 1, 0, 0, 2, 2, 0, 2, 0, 2, 0, 2, 2, 0,
       2, 2, 1, 2, 0, 0, 0, 1, 2, 0, 1, 2, 2, 1, 2, 2, 2, 1, 0, 0, 1, 2,
       0, 2, 1, 0, 2, 1, 0, 2, 0, 0, 1, 0, 1, 2, 2, 2, 0, 2, 0, 2, 2, 2,
       0, 2, 2, 0, 2, 2, 2, 0, 2, 0, 2, 2, 2, 2, 0, 0, 0, 1, 0, 0, 2, 0,
       2, 1, 2, 2, 2, 0, 2, 2, 2, 0, 0, 2, 2, 2, 2, 2, 2, 1, 1, 2, 1, 2,
       0, 2, 2, 1, 2, 0, 0, 2, 2, 1, 0, 1, 2, 2, 0, 2, 2, 1, 2, 2, 0, 2,
       2, 2, 2, 0, 0, 1, 2, 1, 1, 0, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1,
       2, 2, 2, 1, 2, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 2, 1, 1, 0, 2, 1,
       0, 1, 1, 1, 2, 0, 2, 0, 1, 1, 1, 0, 2, 2, 1, 2, 1, 0, 2, 1, 0, 2,
       1, 2, 2, 1, 1, 0, 2, 2, 1, 2, 0, 1, 2, 1, 1, 0, 1, 2, 1, 2, 2, 0,
       1, 2, 2, 2, 1, 2, 2, 1, 2, 1, 1, 2, 1, 2, 2, 1, 1, 2, 2, 0, 1, 2,
    

In [8]:
tf.data.experimental.AUTOTUNE

-1

In [63]:
record_groups

/home/csci5980/piehl008/.conda/envs/wavenet/lib/python3.7/site-packages/jupyter_client/jsonutil.py:66: DeprecationWarning: Interpreting naive datetime as local 2020-04-29 19:32:58.230135. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)


[Int64Index([114508, 143133, 128896], dtype='int64', name='track_id'),
 Int64Index([111317, 77225], dtype='int64', name='track_id'),
 Int64Index([85211, 145005], dtype='int64', name='track_id'),
 Int64Index([59223, 3541], dtype='int64', name='track_id'),
 Int64Index([47728, 131650], dtype='int64', name='track_id'),
 Int64Index([32862, 111107], dtype='int64', name='track_id'),
 Int64Index([81836, 87065], dtype='int64', name='track_id'),
 Int64Index([81333, 69609], dtype='int64', name='track_id'),
 Int64Index([5004, 82941], dtype='int64', name='track_id'),
 Int64Index([96094, 78509], dtype='int64', name='track_id'),
 Int64Index([46198, 58347], dtype='int64', name='track_id'),
 Int64Index([22051, 71520], dtype='int64', name='track_id'),
 Int64Index([82656, 68900], dtype='int64', name='track_id'),
 Int64Index([44972, 43301], dtype='int64', name='track_id'),
 Int64Index([84174, 64455], dtype='int64', name='track_id'),
 Int64Index([87022, 138602], dtype='int64', name='track_id'),
 Int64Index

In [3]:
parsed_dataset

<MapDataset shapes: ((), (None, 44100, 1), (None, 44100, 1)), types: (tf.int64, tf.int64, tf.int64)>

In [53]:
for el in dataset:
    print(tf.io.serialize_tensor(el[0]).numpy())
    feature = {'genre_code': _int64_feature(el[0])}
    print(tf.train.Example(features=tf.train.Features(feature=feature)).SerializeToString())
    break

b'\x08\x03\x12\x00"\x04\x02\x00\x00\x00'
b'\n\x15\n\x13\n\ngenre_code\x12\x05\x1a\x03\n\x01\x02'


/home/csci5980/piehl008/.conda/envs/wavenet/lib/python3.7/site-packages/jupyter_client/jsonutil.py:66: DeprecationWarning: Interpreting naive datetime as local 2020-04-29 19:28:37.122818. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)
